In [4]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

start_date = '2023-05-31'
end_date = '2023-06-30'
store_id = '985'

query = f"""
select qfo.source_store_id,
	 ds.store_name,
	 ds.sale_obj_name,
       sum(qfo.cnt_wares_pos_detail)                                                             as "Кол-во товаров в чеках", -- "Количество товаров в чеке"
       sum(qfo.order_sum_with_vat)                                                               as "Выручка, тг с НДС",
       sum(qfo.order_sum_without_vat)                                                            as "Выручка, тг без НДС",
       sum(qfo.is_sale_order)                                                                    as "Количество чеков"
from dwh_data.qs_fact_order qfo
left join dwh_data.dim_store ds on ds.source_store_id = qfo.source_store_id and current_date between ds.datestart and ds.dateend
where date(qfo.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
        and qfo."source_store_id"::int > 0 and ds."store_name" like '%Торговый%'
group by qfo.source_store_id, ds.store_name, ds.sale_obj_name
     union all
select   qfog.source_store_id,
	   ds.store_name,
	   ds.sale_obj_name,
         sum(qfog.count_wares)                            as quantity,
         sum(qfog.sum_sale_vat)                           as order_sum_with_vat,
         sum(qfog.sum_sale_no_vat)                        as order_sum_without_vat,
         sum(qfog.is_sale_order)                          as sale_order_cnt
from dwh_data.qs_fact_order_gross qfog
left join dwh_data.dim_store ds on ds.source_store_id = qfog.source_store_id and current_date between ds.datestart and ds.dateend
where date(qfog.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
      and qfog."source_store_id"::int > 0 and ds."store_name" like '%Торговый%'
group by qfog.source_store_id, ds.store_name, ds.sale_obj_name
order by source_store_id;
"""

cur.execute(query)

print('Executed')

df = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [5]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

query = f"""
SELECT
      q.source_store_id::int                                                                    as source_store_id,
      Sum(q."sum_order_wares_vat_nr")                                                           as "Оборот, тг с НДС",
      Sum(q."sum_order_wares_without_vat_nr")                                                   as "Оборот, тг без НДС",
      Sum(q."cost_price_base_t")                                                                as "Себестоимость",
      Sum(q."cost_price_base_t") - Sum(q."cost_price_base_t") *
         (Sum(q."sum_order_wares_vat_nr") - Sum(q."sum_order_wares_without_vat_nr"))
         / decode(Sum(q."sum_order_wares_vat_nr"), 0, null, Sum(q."sum_order_wares_vat_nr"))    as "Бухгалтерская себестоимость"
   FROM "dwh_data"."qs_fact_sales" q
   left join dwh_data.dim_store ds on ds.source_store_id = q.source_store_id and current_date between ds.datestart and ds.dateend
   WHERE date(q.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
    and q."source_store_id"::int > 0  and ds."store_name" like '%Торговый%'
   GROUP BY
      q."source_store_id"
 union all
    SELECT
          sa.source_store_id::int                          as source_store_id,
          Sum("sum_sale")                                  as sum_order_with_vat,
          Sum(sa."sum_sale_no_vat")                        as sum_order_without_vat,
          Sum("sum_coming")                                as cost_price_with_vat ,
          Sum(sa."sum_coming_no_vat")                      as cost_price_without_vat
    FROM "dwh_data"."qs_fact_sales_gross" sa
    left join dwh_data.dim_store ds on ds.source_store_id = sa.source_store_id and current_date between ds.datestart and ds.dateend
    where date(sa.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
   and sa.source_store_id::int > 0  and ds."store_name" like '%Торговый%'
    GROUP BY
        sa.source_store_id
    order by source_store_id;
"""

cur.execute(query)

print('Executed')

df1 = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [6]:
df.columns = ["Номер филиала", "Название филиала", "Название компании", "Количество товаров в чеке", "Выручка, тг с НДС", "Выручка, тг без НДС", "Количество чеков"]
df

,Номер филиала,Название филиала,Название компании,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1058,4620773.23,4125690.39,38
1,4,Торговый зал АФ №1,Алматинский филиал №1,1297961,1107567548.00,988892733.97,167986
2,96,Торговый зал АФ №2,Алматинский филиал №2,202,3599241.65,3213608.62,7
3,96,Торговый зал АФ №2,Алматинский филиал №2,1463979,1324785550.00,1182836423.01,186718
4,102,Торговый зал АФ №3,Алматинский филиал №3,303911,227054471.00,202725984.43,61832
...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,20,791775.00,706941.96,1


In [7]:
df1.columns = ["Номер филиала", "Оборот, тг с НДС", "Оборот, тг без НДС", "Себестоимость", "Бухгалтерская себестоимость"]
df1

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,4620773.11200000000000000000000000000000050633...,4125690.27857142857142857142857142857142902351...,4474065.34700519390379987107776989203657171327...,3996021.98653839767124988489086597503265331542...
1,4,1107602443.44999999985065220000000000000000000...,988927615.937452901265999761359999999999980939...,1026635115.48948021306138040080547302511458825...,916635588.159493864275248799163448522528442122...
2,96,1324822948.32300000057756980000000000000000000...,1182875360.29105829941137231749714285714282898...,1221109682.84396865360328722783082943939028375...,1090274408.27270506952392863162663025082269529...
3,96,3599241.63600000000000000000000000000000237494...,3213608.60357142857142857142857142857143069191...,3542664.02889189143996418410620719226247769963...,3166746.22741491021425373580911356452006937467...
4,102,227062284.507999999993745070000000000000000000...,202733951.887842287663823566785714285714283599...,216256516.427958449256635966248765047152569543...,193085955.652813032034934580654554633777379598...
...,...,...,...,...,...
277,2889,14392566.6320000000000000000000000000000000000...,12850505.9214256126422114028571428571428574377...,11340192.0118771360277214808112320152013119581...,10125171.4391736830233294959908743711874831043...
278,2970,24470188.4530000000000000000000000000000000000...,21848382.5473204261990434600000000000000000094...,19205898.3128293344705217344660670058948762484...,17148123.4935968333317096768912796102495998437...
279,2975,20287642.9890000000004414000000000000000000000...,18113873.3544623003269835314285714285714295516...,16949671.0167385889873287237766721945826423459...,15133556.6365926380156764496903197867742671841...
280,2989,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...


In [8]:
df2 = df1.copy()
for i in df2['Номер филиала'].unique():
    print(df2[df2['Номер филиала'] == i].index)
    ids = df2[df2['Номер филиала'] == i].index
    # print(df2.loc[ids, df2.columns[1:]].sum())
    df2.loc[ids[0], df2.columns[1:]] = df2.loc[ids, df2.columns[1:]].sum()

Index([0, 1], dtype='int64')
Index([2, 3], dtype='int64')
Index([4, 5], dtype='int64')
Index([6, 7], dtype='int64')
Index([8, 9], dtype='int64')
Index([10, 11], dtype='int64')
Index([12], dtype='int64')
Index([13, 14], dtype='int64')
Index([15, 16], dtype='int64')
Index([17, 18], dtype='int64')
Index([19, 20], dtype='int64')
Index([21, 22], dtype='int64')
Index([23], dtype='int64')
Index([24, 25], dtype='int64')
Index([26], dtype='int64')
Index([27, 28], dtype='int64')
Index([29, 30], dtype='int64')
Index([31, 32], dtype='int64')
Index([33, 34], dtype='int64')
Index([35, 36], dtype='int64')
Index([37], dtype='int64')
Index([38], dtype='int64')
Index([39], dtype='int64')
Index([40, 41], dtype='int64')
Index([42], dtype='int64')
Index([43, 44], dtype='int64')
Index([45], dtype='int64')
Index([46], dtype='int64')
Index([47, 48], dtype='int64')
Index([49, 50], dtype='int64')
Index([51, 52], dtype='int64')
Index([53], dtype='int64')
Index([54, 55], dtype='int64')
Index([56], dtype='int64')


In [9]:
df1

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,4620773.11200000000000000000000000000000050633...,4125690.27857142857142857142857142857142902351...,4474065.34700519390379987107776989203657171327...,3996021.98653839767124988489086597503265331542...
1,4,1107602443.44999999985065220000000000000000000...,988927615.937452901265999761359999999999980939...,1026635115.48948021306138040080547302511458825...,916635588.159493864275248799163448522528442122...
2,96,1324822948.32300000057756980000000000000000000...,1182875360.29105829941137231749714285714282898...,1221109682.84396865360328722783082943939028375...,1090274408.27270506952392863162663025082269529...
3,96,3599241.63600000000000000000000000000000237494...,3213608.60357142857142857142857142857143069191...,3542664.02889189143996418410620719226247769963...,3166746.22741491021425373580911356452006937467...
4,102,227062284.507999999993745070000000000000000000...,202733951.887842287663823566785714285714283599...,216256516.427958449256635966248765047152569543...,193085955.652813032034934580654554633777379598...
...,...,...,...,...,...
277,2889,14392566.6320000000000000000000000000000000000...,12850505.9214256126422114028571428571428574377...,11340192.0118771360277214808112320152013119581...,10125171.4391736830233294959908743711874831043...
278,2970,24470188.4530000000000000000000000000000000000...,21848382.5473204261990434600000000000000000094...,19205898.3128293344705217344660670058948762484...,17148123.4935968333317096768912796102495998437...
279,2975,20287642.9890000000004414000000000000000000000...,18113873.3544623003269835314285714285714295516...,16949671.0167385889873287237766721945826423459...,15133556.6365926380156764496903197867742671841...
280,2989,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...


In [21]:
df2

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928
1,4,1123379963.81399999973863970000000000000000000...,1003011549.23703488965819413242857142857144278...,1030680594.30727868904444421862131647315669821...,920244772.885994759307399892562363841853388926...
2,96,1339502056.353000000223722460,1195982530.812155182793964708,1226697036.771772817281448659,1095269826.842337979265899657
3,96,1949488.74800000000000000000000000000000015639...,1740614.95357142857142857142857142857142871107...,1712122.67828342327111952111557085183239691294...,1534622.60706278313492814385318826056464010084...
4,102,240085507.5420000000654364000,214362060.3053244444117981321,229786116.6436187471497237116,205169820.9856576170813540367
...,...,...,...,...,...
279,2889,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10330744.5798412264322759166842375976900297940...,9223879.08914316209735512821603190201112898813...
280,2970,0E-42,0E-42,0E-42,0
281,2975,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475411.5835863307248426509488403356875552909...,14710172.5918456181879470548464141275748106842...
282,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [10]:
df2 = df2.loc[~df2['Номер филиала'].duplicated(keep='first')]

In [11]:
df2

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,1112223216.561999999850652200,993053306.2160243298374283328,1031109180.836485406965180272,920631610.1460322619464986841
2,96,1328422189.959000000577569800,1186088968.894629727982800889,1224652346.872860545043251412,1093441154.500119979738182367
4,102,229225781.5079999999937450700,204665645.6378422876638235668,218600066.2990088481046359662,195178410.8948223167206488664
6,105,638354430.1680000003179801900,570285820.8213641984298588943,600850279.9483025075052908063,536785897.1319939341641482583
8,116,482270568.1219999998680444200,430725059.6803167550039824558,467350606.6398430483718433698,417417166.3008108779060985943
...,...,...,...,...,...
277,2889,14392566.6320000000000000000000000000000000000...,12850505.9214256126422114028571428571428574377...,11340192.0118771360277214808112320152013119581...,10125171.4391736830233294959908743711874831043...
278,2970,24470188.4530000000000000000000000000000000000...,21848382.5473204261990434600000000000000000094...,19205898.3128293344705217344660670058948762484...,17148123.4935968333317096768912796102495998437...
279,2975,20287642.9890000000004414000000000000000000000...,18113873.3544623003269835314285714285714295516...,16949671.0167385889873287237766721945826423459...,15133556.6365926380156764496903197867742671841...
280,2989,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...


In [12]:
len(df2)

240

In [13]:
df3 = df.copy()
for i in df3['Номер филиала'].unique():
    print(i, df3[df3['Номер филиала'] == i].index)
    ids = df3[df3['Номер филиала'] == i].index
    # print(df3.loc[ids, df3.columns[2:]].sum())
    df3.loc[ids[0], df3.columns[3:]] = df3.loc[ids, df3.columns[3:]].sum()
    try:
        df3 = df3.drop([ids[1]])
    except:
        pass

4 Index([0, 1], dtype='int64')
96 Index([2, 3], dtype='int64')
102 Index([4, 5], dtype='int64')
105 Index([6, 7], dtype='int64')
116 Index([8, 9], dtype='int64')
140 Index([10, 11], dtype='int64')
197 Index([12], dtype='int64')
220 Index([13, 14], dtype='int64')
280 Index([15, 16], dtype='int64')
320 Index([17, 18], dtype='int64')
360 Index([19, 20], dtype='int64')
401 Index([21, 22], dtype='int64')
421 Index([23], dtype='int64')
441 Index([24, 25], dtype='int64')
444 Index([26], dtype='int64')
461 Index([27, 28], dtype='int64')
501 Index([29, 30], dtype='int64')
521 Index([31, 32], dtype='int64')
541 Index([33, 34], dtype='int64')
542 Index([35, 36], dtype='int64')
561 Index([37], dtype='int64')
601 Index([38], dtype='int64')
602 Index([39], dtype='int64')
631 Index([40, 41], dtype='int64')
632 Index([42], dtype='int64')
633 Index([43, 44], dtype='int64')
634 Index([45], dtype='int64')
635 Index([46], dtype='int64')
641 Index([47, 48], dtype='int64')
681 Index([49, 50], dtype='int64')

In [14]:
df3

,Номер филиала,Название филиала,Название компании,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1299019,1112188321.23,993018424.36,168024
2,96,Торговый зал АФ №2,Алматинский филиал №2,1464181,1328384791.65,1186050031.63,186725
4,102,Торговый зал АФ №3,Алматинский филиал №3,303979,229217968.00,204657678.18,61841
6,105,Торговый зал КФ №1,Карагандинский филиал №1,786883,638332929.68,569937148.56,115540
8,116,Торговый зал АСФ №1,Астанинский филиал №1,403042,482260300.73,430588279.32,52598
...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,20,791775.00,706941.96,1


In [15]:
df

,Номер филиала,Название филиала,Название компании,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1058,4620773.23,4125690.39,38
1,4,Торговый зал АФ №1,Алматинский филиал №1,1297961,1107567548.00,988892733.97,167986
2,96,Торговый зал АФ №2,Алматинский филиал №2,202,3599241.65,3213608.62,7
3,96,Торговый зал АФ №2,Алматинский филиал №2,1463979,1324785550.00,1182836423.01,186718
4,102,Торговый зал АФ №3,Алматинский филиал №3,303911,227054471.00,202725984.43,61832
...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,20,791775.00,706941.96,1


In [16]:
df3

,Номер филиала,Название филиала,Название компании,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1299019,1112188321.23,993018424.36,168024
2,96,Торговый зал АФ №2,Алматинский филиал №2,1464181,1328384791.65,1186050031.63,186725
4,102,Торговый зал АФ №3,Алматинский филиал №3,303979,229217968.00,204657678.18,61841
6,105,Торговый зал КФ №1,Карагандинский филиал №1,786883,638332929.68,569937148.56,115540
8,116,Торговый зал АСФ №1,Астанинский филиал №1,403042,482260300.73,430588279.32,52598
...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,20,791775.00,706941.96,1


In [17]:
df3

,Номер филиала,Название филиала,Название компании,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1299019,1112188321.23,993018424.36,168024
2,96,Торговый зал АФ №2,Алматинский филиал №2,1464181,1328384791.65,1186050031.63,186725
4,102,Торговый зал АФ №3,Алматинский филиал №3,303979,229217968.00,204657678.18,61841
6,105,Торговый зал КФ №1,Карагандинский филиал №1,786883,638332929.68,569937148.56,115540
8,116,Торговый зал АСФ №1,Астанинский филиал №1,403042,482260300.73,430588279.32,52598
...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,20,791775.00,706941.96,1


In [18]:
df4 = pd.concat([df2, df3], join='inner', axis=1)
df4.columns = ['Номер филиала', 'Оборот, тг с НДС', 'Оборот, тг без НДС',
       'Себестоимость', 'Бухгалтерская себестоимость', 'Номер филиала1',
       'Название филиала', 'Название компании', 'Количество товаров в чеке', 'Выручка, тг с НДС',
       'Выручка, тг без НДС', 'Количество чеков']
df4.insert(1, 'Название филиала', df4.pop('Название филиала'))
df4.insert(2, 'Название компании', df4.pop('Название компании'))
df4

,Номер филиала,Название филиала,Название компании,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Номер филиала1,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1112223216.561999999850652200,993053306.2160243298374283328,1031109180.836485406965180272,920631610.1460322619464986841,4,1299019,1112188321.23,993018424.36,168024
2,96,Торговый зал АФ №2,Алматинский филиал №2,1328422189.959000000577569800,1186088968.894629727982800889,1224652346.872860545043251412,1093441154.500119979738182367,96,1464181,1328384791.65,1186050031.63,186725
4,102,Торговый зал АФ №3,Алматинский филиал №3,229225781.5079999999937450700,204665645.6378422876638235668,218600066.2990088481046359662,195178410.8948223167206488664,102,303979,229217968.00,204657678.18,61841
6,105,Торговый зал КФ №1,Карагандинский филиал №1,638354430.1680000003179801900,570285820.8213641984298588943,600850279.9483025075052908063,536785897.1319939341641482583,105,786883,638332929.68,569937148.56,115540
8,116,Торговый зал АСФ №1,Астанинский филиал №1,482270568.1219999998680444200,430725059.6803167550039824558,467350606.6398430483718433698,417417166.3008108779060985943,116,403042,482260300.73,430588279.32,52598
...,...,...,...,...,...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,14392566.6320000000000000000000000000000000000...,12850505.9214256126422114028571428571428574377...,11340192.0118771360277214808112320152013119581...,10125171.4391736830233294959908743711874831043...,2889,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,24470188.4530000000000000000000000000000000000...,21848382.5473204261990434600000000000000000094...,19205898.3128293344705217344660670058948762484...,17148123.4935968333317096768912796102495998437...,2970,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,20287642.9890000000004414000000000000000000000...,18113873.3544623003269835314285714285714295516...,16949671.0167385889873287237766721945826423459...,15133556.6365926380156764496903197867742671841...,2975,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,2989,20,791775.00,706941.96,1


In [19]:
df4 = df4.drop(['Номер филиала1'], axis=1)
df4

,Номер филиала,Название филиала,Название компании,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1112223216.561999999850652200,993053306.2160243298374283328,1031109180.836485406965180272,920631610.1460322619464986841,1299019,1112188321.23,993018424.36,168024
2,96,Торговый зал АФ №2,Алматинский филиал №2,1328422189.959000000577569800,1186088968.894629727982800889,1224652346.872860545043251412,1093441154.500119979738182367,1464181,1328384791.65,1186050031.63,186725
4,102,Торговый зал АФ №3,Алматинский филиал №3,229225781.5079999999937450700,204665645.6378422876638235668,218600066.2990088481046359662,195178410.8948223167206488664,303979,229217968.00,204657678.18,61841
6,105,Торговый зал КФ №1,Карагандинский филиал №1,638354430.1680000003179801900,570285820.8213641984298588943,600850279.9483025075052908063,536785897.1319939341641482583,786883,638332929.68,569937148.56,115540
8,116,Торговый зал АСФ №1,Астанинский филиал №1,482270568.1219999998680444200,430725059.6803167550039824558,467350606.6398430483718433698,417417166.3008108779060985943,403042,482260300.73,430588279.32,52598
...,...,...,...,...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,14392566.6320000000000000000000000000000000000...,12850505.9214256126422114028571428571428574377...,11340192.0118771360277214808112320152013119581...,10125171.4391736830233294959908743711874831043...,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,24470188.4530000000000000000000000000000000000...,21848382.5473204261990434600000000000000000094...,19205898.3128293344705217344660670058948762484...,17148123.4935968333317096768912796102495998437...,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,20287642.9890000000004414000000000000000000000...,18113873.3544623003269835314285714285714295516...,16949671.0167385889873287237766721945826423459...,15133556.6365926380156764496903197867742671841...,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,20,791775.00,706941.96,1


In [20]:
df4

,Номер филиала,Название филиала,Название компании,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,Алматинский филиал №1,1112223216.561999999850652200,993053306.2160243298374283328,1031109180.836485406965180272,920631610.1460322619464986841,1299019,1112188321.23,993018424.36,168024
2,96,Торговый зал АФ №2,Алматинский филиал №2,1328422189.959000000577569800,1186088968.894629727982800889,1224652346.872860545043251412,1093441154.500119979738182367,1464181,1328384791.65,1186050031.63,186725
4,102,Торговый зал АФ №3,Алматинский филиал №3,229225781.5079999999937450700,204665645.6378422876638235668,218600066.2990088481046359662,195178410.8948223167206488664,303979,229217968.00,204657678.18,61841
6,105,Торговый зал КФ №1,Карагандинский филиал №1,638354430.1680000003179801900,570285820.8213641984298588943,600850279.9483025075052908063,536785897.1319939341641482583,786883,638332929.68,569937148.56,115540
8,116,Торговый зал АСФ №1,Астанинский филиал №1,482270568.1219999998680444200,430725059.6803167550039824558,467350606.6398430483718433698,417417166.3008108779060985943,403042,482260300.73,430588279.32,52598
...,...,...,...,...,...,...,...,...,...,...,...
277,2889,Торговый зал АСФ №81,Астанинский филиал №81,14392566.6320000000000000000000000000000000000...,12850505.9214256126422114028571428571428574377...,11340192.0118771360277214808112320152013119581...,10125171.4391736830233294959908743711874831043...,24604,14391482.00,12849420.81,8126
278,2970,Торговый зал АФ №86,Алматинский филиал №86,24470188.4530000000000000000000000000000000000...,21848382.5473204261990434600000000000000000094...,19205898.3128293344705217344660670058948762484...,17148123.4935968333317096768912796102495998437...,43739,24468923.00,21847117.80,14245
279,2975,Торговый зал АСФ №82,Астанинский филиал №82,20287642.9890000000004414000000000000000000000...,18113873.3544623003269835314285714285714295516...,16949671.0167385889873287237766721945826423459...,15133556.6365926380156764496903197867742671841...,35157,20286247.00,18112478.85,10848
280,2989,Торговый зал СТМ 5АСФ,РЦ №1 филиал в г.Астана,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,791775.000000000287040000000000000000000000000...,706941.964285714542000000000000000000000000000...,20,791775.00,706941.96,1


In [42]:
import openpyxl
import os
import datetime

for i in range(len(df4)):
    book = openpyxl.load_workbook('samples\\2023-04-30_2023-05-31_1583_4.xlsx')
    sheet = book.active
    
    try:
        start_date = start_date.split('-')[2] + '.' + start_date.split('-')[1] + '.' + start_date.split('-')[0]
        end_date = end_date.split('-')[2] + '.' + end_date.split('-')[1] + '.' + end_date.split('-')[0]
    except:
        ...

    sheet['D2'] = start_date
    sheet['G2'] = end_date
    sheet['B4'].value = str(datetime.datetime.now().strftime("%d.%m.%y %H:%M:%S"))
    sheet['B7'].value = df4['Название филиала'].iloc[i]
    sheet['A11'].value = start_date + '-' + end_date
    sheet['B11'].value = df4['Название филиала'].iloc[i]
    sheet['C11'].value = df4['Название филиала'].iloc[i]
    sheet['D11'].value = df4['Количество чеков'].iloc[i]
    sheet['E11'].value = df4['Выручка, тг с НДС'].iloc[i]
    sheet['F11'].value = df4['Выручка, тг без НДС'].iloc[i]
    sheet['G11'].value = df4['Оборот, тг с НДС'].iloc[i]
    sheet['H11'].value = df4['Оборот, тг без НДС'].iloc[i]
    sheet['I11'].value = df4['Себестоимость'].iloc[i]
    sheet['J11'].value = df4['Бухгалтерская себестоимость'].iloc[i]
    sheet['K11'].value = float(df4['Оборот, тг с НДС'].iloc[i]) - float(df4['Себестоимость'].iloc[i])
    try:
        sheet['L11'].value = float(df4['Оборот, тг с НДС'].iloc[i]) / float(df4['Себестоимость'].iloc[i]) - 1
    except:
        sheet['L11'].value = 0
    sheet['L11'].number_format = '0.00%'
    sheet['M11'].value = df4['Количество товаров в чеке'].iloc[i]

    sheet['D12'].value = sheet['D11'].value
    sheet['E12'].value = sheet['E11'].value
    sheet['F12'].value = sheet['F11'].value
    sheet['G12'].value = sheet['G11'].value
    sheet['H12'].value = sheet['H11'].value
    sheet['I12'].value = sheet['I11'].value
    sheet['J12'].value = sheet['J11'].value
    sheet['K12'].value = sheet['K11'].value
    sheet['L12'].value = sheet['L11'].value
    sheet['M12'].value = sheet['M11'].value
    
    try:
        os.makedirs('samples\\1583_1')
    except:
        pass
    book.save(f'samples\\1583_1\\{df4["Название филиала"].iloc[i]}_1583.xlsx')
    print('Saved', df4['Название филиала'].iloc[i])

Saved Торговый зал АФ №1
Saved Торговый зал АФ №2
Saved Торговый зал АФ №3
Saved Торговый зал КФ №1
Saved Торговый зал АСФ №1
Saved Торговый зал АФ №4
Saved Торговый зал АФ №5
Saved Торговый зал АФ №6
Saved Торговый зал АФ №7
Saved Торговый зал ШФ №1
Saved Торговый зал АФ №8
Saved Торговый зал АФ №10
Saved Торговый зал АФ №9
Saved Торговый зал АФ №11
Saved Торговый зал АФ №12
Saved Торговый зал ШФ №2
Saved Торговый зал ТКФ №1
Saved Торговый зал АФ №14
Saved Торговый зал АСФ №2
Saved Торговый зал АСФ №3
Saved Торговый зал ТКФ №2
Saved Торговый зал АФ №16
Saved Торговый зал АФ №15
Saved Торговый зал АФ №19
Saved Торговый зал АФ №17
Saved Торговый зал ФКС №1
Saved Торговый зал АФ №20
Saved Торговый зал АФ №18
Saved Торговый зал ШФ №4
Saved Торговый зал КФ №2
Saved Торговый зал АФ №21
Saved Торговый зал ППФ №1
Saved Торговый зал АФ №22
Saved Торговый зал АФ №23
Saved Торговый зал ШФ №3
Saved Торговый зал АФ №24
Saved Торговый зал АСФ №4
Saved Торговый зал АСФ №5
Saved Торговый зал АФ №25
S

In [133]:
df4


,Номер филиала,Название филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,0E-42,0E-42,0E-42,0,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,44,1098237.00,980568.75,2


In [131]:
book.save('samples\\kekus.xlsx')

In [100]:
df4

,Номер филиала,Название филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,Торговый зал АФ №1,1129646973.640999999738639700,1008607093.725427746801051275,1036605822.604656972674440610,925545956.3664756799894164928,1324812,1129610584.89,1008570720.08,169503
2,96,Торговый зал АФ №2,1339502056.353000000223722460,1195982530.812155182793964708,1226603063.203776478081448659,1095185921.974602662904594268,1467812,1339464146.76,1195942150.23,185308
4,102,Торговый зал АФ №3,240085507.5420000000654364000,214362060.3053244444117981321,229709513.5536203471497237116,205101425.3695876276428354920,300259,240077375.00,214353949.81,58213
6,105,Торговый зал КФ №1,578929044.2019999998617525300,516899921.4045888748025048478,537354539.5527312199941891141,479786066.7917906438582291571,712262,578917761.74,516888589.54,104683
8,116,Торговый зал АСФ №1,490101131.4837000005979525100,437708737.3946928404883147482,478358695.8100691511677571746,427221655.2541790727437284717,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...,...,...,...,...,...
279,2889,Торговый зал АСФ №81,13033266.6000000000000000000000000000000000000...,11636845.1785704318137249857142857142857144045...,10319447.7298412264322759166842375976900297940...,9213792.61592887724702950999343938414063376072...,22454,13032280.00,11635858.71,6966
280,2970,Торговый зал АФ №86,0E-42,0E-42,0E-42,0,2,0.00,0.00,2
281,2975,Торговый зал АСФ №82,20024950.8999999999798600000000000000000000000...,17879400.6078549693236739900000000000000008792...,16475507.3335863307248426509488403356875552909...,14710258.0828221879289152818183369708204887593...,33592,20023547.00,17877998.41,10315
282,2989,Торговый зал СТМ 5АСФ,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,44,1098237.00,980568.75,2


In [ ]:
for i in df2['Номер филиала'].unique():
    print(df2[df2['Номер филиала'] == i].index)
    ids = df2[df2['Номер филиала'] == i].index
    print(df2.loc[ids, df2.columns[1:]].sum())
    df2.loc[ids[0], df2.columns[1:]] = df2.loc[ids, df2.columns[1:]].sum()

Index([0], dtype='int64')
Количество товаров в чеке                                                  11783
Выручка, тг с НДС                                                     7525031.00
Выручка, тг без НДС                                                   6718734.66
Количество чеков                                                            2589
Оборот, тг с НДС               82133802811.8219999545656379960000000000000000...
Оборот, тг без НДС             73347493095.2100944447412386556031746031742836...
Себестоимость                  57494025081.6455697745459859511759702966979874...
Бухгалтерская себестоимость    51343569435.7603851262871592390984197815968943...
dtype: object
Index([1, 2], dtype='int64')
Количество товаров в чеке                          132801427
Выручка, тг с НДС              82365149978.53000000008417696
Выручка, тг без НДС            73521297372.96154761912279914
Количество чеков                                    13344863
Оборот, тг с НДС               57053341